In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import seaborn as sns
from itertools import permutations 



# Problem #1 : cvtColor

In [2]:
in_img = cv2.imread("./MeeseeksHQ.png")
gray_img = cv2.cvtColor(in_img,cv2.COLOR_BGR2GRAY)
gray_3_channel = np.stack((gray_img,)*3, axis=-1)
out_img = cv2.hconcat([in_img,gray_3_channel])
cv2.imwrite("./outputs/P1-Gray using cvtColor.png",out_img)

True

# Problem #2 : mr meeskees detection by color

In [3]:
in_img = cv2.imread("./mr meeskees.png")
b_ranges = list(range(49,96))
g_ranges = list(range(88,142))
r_ranges = list(range(99,146))

mask = np.zeros_like(in_img,dtype=np.uint8)
for row in range(in_img.shape[0]):
    for col in range(in_img.shape[1]):
        b,g,r = in_img[row,col]
        if b in r_ranges and g in g_ranges and r in b_ranges:
            mask[row,col] = [255,255,255]
            
mask = cv2.medianBlur(mask,3)
out = cv2.vconcat([mask,cv2.bitwise_and(mask,in_img)])
cv2.imwrite("./outputs/P2-mr meeskees detetion.png",out)

True

# Problem #3: Crop/Register/Colorize

### Part one

In [27]:
def SplitCropper(img):
    UpBotT = 12
    RightLeftT = 25
    img = img[UpBotT:,:]
    img = img[:img.shape[0]-UpBotT,:]
    img = img[:,RightLeftT:]
    img = img[:,:img.shape[1]-RightLeftT]
    
    H,_ = img.shape
    Channels = []
    Channels.append(img[:H//3,:])
    Channels.append(img[H//3:H//3*2,:])
    Channels.append(img[H//3*2:H-1,:])
    return Channels

### Part Two

In [28]:
def ColorizeRGB(channels,Name="tmp.jpg"):
    perm = permutations([0,1, 2]) 
    imgs = []
    for p in perm:
        new_img = np.stack([channels[p[0]],channels[p[1]],channels[p[2]]],axis=2)
        imgs.append(new_img)
        
    H1 = cv2.hconcat(imgs[:2])
    H2 = cv2.hconcat(imgs[2:4])
    H3 = cv2.hconcat(imgs[4:])
    All = cv2.vconcat([H1,H2,H3])
    All = cv2.cvtColor(All,cv2.COLOR_RGB2BGR)
    cv2.imwrite(Name,All)
    return imgs[-1]

### Part Three

In [31]:
def SortKey(item):
    return item[0]

def DeBlur(img_RGB):
    H,W,_ = img_RGB.shape
    Pad = 300
    base = np.zeros((Pad+H,Pad+W,3))    
    Pad = Pad//2
    laps = []
    
    for i in range(3):
        tmp = img_RGB[:,:,i].copy()
        tmp = cv2.Laplacian(tmp,cv2.CV_64F)
        laps.append(tmp)
        
    SearchL = 75
    
    # Constant Red Channel
    base[Pad:Pad+H,Pad:Pad+W,0] = laps[0]
    
    #register
    offsets = []
    for index in [1,2]:
        diffs = []
        for row in range(Pad-SearchL,Pad+SearchL):
            for col in range(Pad-SearchL,Pad+SearchL):
                base[row:row+H,col:col+W,index] = laps[index]
                dif = ((base[:,:,0]-base[:,:,index])**2).sum()
                diffs.append([dif,row,col])
                base[:,:,index] = 0
                
        diffs.sort(key=SortKey)
        offset_ = diffs[0]
        offsets.append(offset_)
    
    final_img = np.zeros_like(base,dtype=np.uint8)
    final_img[Pad:Pad+H,Pad:Pad+W,0] = img_RGB[:,:,0]
    for index in range(2):
        row,col = offsets[index][1:]
        final_img[row:row+H,col:col+W,index+1] = img_RGB[:,:,index+1]
            
    final_img = final_img[Pad:Pad+H,Pad:Pad+W,:]
    
    return final_img
            
    
list_imgs = ["./01.jpg","02.jpg","03.jpg","04.jpg"]
I = 0
for img_dir in list_imgs:
    I+=1 
    img = cv2.imread(img_dir,0)
    img_org = img.copy()
    channels = SplitCropper(img)
    
    RGB_img = ColorizeRGB(channels,"./outputs/P3-Color channel try-{0}.jpg".format(I))
    deblur = DeBlur(RGB_img)
    deblur_ = cv2.cvtColor(deblur,cv2.COLOR_RGB2BGR)
    cv2.imwrite("./outputs/P3-Deblur-{0}.jpg".format(I),deblur_)
    
    comp = cv2.vconcat([RGB_img,deblur])
    comp_ = cv2.cvtColor(comp,cv2.COLOR_RGB2BGR)
    cv2.imwrite("./outputs/P3-Compare-{0}.jpg".format(I),comp_)



# Problem #4: Canny

### Primary Canny

In [37]:
in_img = cv2.imread("./edge.jpg",0)
DetectedEdges = cv2.Canny(in_img,100,200) # parameter by https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html 
Compare = cv2.hconcat([DetectedEdges,in_img])
cv2.imwrite("./outputs/P4-Canny out-compare.jpg",Compare)


True

### Remove extra edges

In [50]:
# Median Filtering
for k in [3,5,7,9]:
    in_img = cv2.imread("./edge.jpg",0)
    img_f = cv2.medianBlur(in_img,k)
    DetectedEdges = cv2.Canny(img_f,100,200)
    Compare = cv2.hconcat([DetectedEdges,img_f,in_img])
    cv2.imwrite("./outputs/P4-Canny out-compare-Median-{0}.jpg".format(k),Compare)
    

# Blur Filtering - avg
for k in [3,5,7,9]:
    in_img = cv2.imread("./edge.jpg",0)
    img_f = cv2.blur(in_img,(k,k))
    DetectedEdges = cv2.Canny(img_f,100,200)
    Compare = cv2.hconcat([DetectedEdges,img_f,in_img])
    cv2.imwrite("./outputs/P4-Canny out-compare-AVG-{0}.jpg".format(k),Compare)
    
    

# Gus Filtering - avg
for k in [3,5,7,9]:
    in_img = cv2.imread("./edge.jpg",0)
    img_f = cv2.GaussianBlur(in_img,(k,k),20)
    DetectedEdges = cv2.Canny(img_f,100,200)
    Compare = cv2.hconcat([DetectedEdges,img_f,in_img])
    cv2.imwrite("./outputs/P4-Canny out-compare-GuassianBlur-{0}.jpg".format(k),Compare)
    

# Bilateral Filtering - avg
for k in [3,5,7,9]:
    in_img = cv2.imread("./edge.jpg",0)
    img_f = cv2.bilateralFilter(in_img,k,95,65)
    DetectedEdges = cv2.Canny(img_f,100,200)
    Compare = cv2.hconcat([DetectedEdges,img_f,in_img])
    cv2.imwrite("./outputs/P4-Canny out-compare-Bilateral-{0}.jpg".format(k),Compare)

# Problem 5 : Histogram

In [67]:
# RGB Channel
img_in = cv2.imread("./P5-in.jpg")
equlize = np.zeros_like(img_in,dtype=np.uint8)
for c in range(3):
    equlize[:,:,c] = cv2.equalizeHist(img_in[:,:,c])
cv2.imwrite("./outputs/P5-result-RGB.jpg",equlize)
com = cv2.vconcat([img_in,equlize])
cv2.imwrite("./outputs/P5-compare-RGB.jpg",com)

# HSV Channel 
img_in = cv2.imread("./P5-in.jpg")
hsv = cv2.cvtColor(img_in,cv2.COLOR_BGR2HSV)
hsv[:,:,2] = cv2.equalizeHist(hsv[:,:,2])
equlize = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

cv2.imwrite("./outputs/P5-result-HSV.jpg",equlize)
com = cv2.vconcat([img_in,equlize])
cv2.imwrite("./outputs/P5-compare-HSV.jpg",com)


True

(3, 256)